# Earth System Simulation - Google Colab Version

This notebook shows how to run the Earth system simulation in Google Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yourusername/earth_system_sim/blob/main/examples/colab_simulation.ipynb)

## 1. Setup and Validation

First, let's set up the environment and validate all components.

In [ ]:
# Install required packages
!pip install torch numpy matplotlib pyyaml tqdm plotly

In [ ]:
# Clone repository
!git clone https://github.com/yourusername/earth_system_sim.git
%cd earth_system_sim

# Install package in development mode
!pip install -e .

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import yaml
from pathlib import Path
import sys
from typing import Dict, List, Tuple, Optional, Union, Any

# Verify imports work correctly
def verify_imports():
    try:
        from scripts.run_simulation import EarthSystemSimulation
        from scripts.visualize_results import create_visualizations
        from src.models.physical import PINN
        from src.models.biosphere import BiospherePolicy
        from src.models.geosphere import GeospherePolicy
        print("All imports successful!")
        return True
    except ImportError as e:
        print(f"Import error: {str(e)}")
        return False

# Verify system configuration
def verify_system():
    print("System Configuration:")
    print(f"Python version: {sys.version}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"NumPy version: {np.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA device: {torch.cuda.get_device_name(0)}")

# Run verifications
verify_system()
if not verify_imports():
    raise RuntimeError("Import verification failed. Please check the setup.")

In [ ]:
# Validate configuration
def validate_config(config: Dict) -> bool:
    """Validate configuration structure and values."""
    required_keys = ['grid_height', 'grid_width', 'physical_system', 
                     'biosphere', 'geosphere', 'integration']
    
    # Check required keys
    for key in required_keys:
        if key not in config:
            print(f"Missing required key: {key}")
            return False
    
    # Validate grid dimensions
    if config['grid_height'] <= 0 or config['grid_width'] <= 0:
        print("Invalid grid dimensions")
        return False
    
    return True

# Load and validate configuration
with open('config/model_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

if not validate_config(config):
    raise ValueError("Configuration validation failed")

print("Configuration validated successfully!")

## 2. Initialize Simulation

Now we'll set up the simulation with proper error handling.

In [ ]:
# Initialize simulation with error handling
def initialize_simulation(config_path: str) -> Any:
    """Initialize simulation with proper error handling."""
    try:
        from scripts.run_simulation import EarthSystemSimulation
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        sim = EarthSystemSimulation(config_path, device)
        return sim
    except Exception as e:
        print(f"Error initializing simulation: {str(e)}")
        raise

# Initialize simulation
sim = initialize_simulation('config/model_config.yaml')
print("Simulation initialized successfully!")

## 3. Run Simulation

Run the simulation with progress tracking and error handling.

In [ ]:
from tqdm.notebook import tqdm

def run_simulation_with_progress(sim: Any, num_steps: int, save_frequency: int) -> Dict:
    """Run simulation with progress tracking and error handling."""
    try:
        print("Starting simulation...")
        trajectory = sim.run_simulation(
            num_steps=num_steps,
            save_frequency=save_frequency
        )
        print("Simulation completed successfully!")
        return trajectory
    except Exception as e:
        print(f"Error during simulation: {str(e)}")
        raise

# Run simulation
trajectory = run_simulation_with_progress(sim, num_steps=50, save_frequency=5)

## 4. Analyze Results

Create visualizations and analyze the simulation results.

In [ ]:
def plot_results(trajectory: Dict):
    """Create basic plots of simulation results."""
    fig, axes = plt.subplots(3, 1, figsize=(12, 12))
    
    # Plot physical system results
    temp_mean = np.mean(trajectory['physical'][..., 0], axis=(1, 2))
    axes[0].plot(temp_mean, 'r-', label='Mean Temperature')
    axes[0].set_title('Physical System')
    axes[0].grid(True)
    axes[0].legend()
    
    # Plot biosphere results
    veg_mean = np.mean(trajectory['biosphere'][..., 0], axis=0)
    axes[1].plot(veg_mean, 'g-', label='Mean Vegetation')
    axes[1].set_title('Biosphere System')
    axes[1].grid(True)
    axes[1].legend()
    
    # Plot geosphere results
    elev_mean = np.mean(trajectory['geosphere'][..., 0], axis=(1, 2))
    axes[2].plot(elev_mean, 'b-', label='Mean Elevation')
    axes[2].set_title('Geosphere System')
    axes[2].grid(True)
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()

# Plot results
try:
    plot_results(trajectory)
except Exception as e:
    print(f"Error plotting results: {str(e)}")

## 5. Save Results

Save the simulation results to Google Drive.

In [ ]:
def save_results(trajectory: Dict, save_path: str):
    """Save simulation results with error handling."""
    try:
        np.savez(
            save_path,
            physical_states=trajectory['physical'],
            biosphere_states=trajectory['biosphere'],
            geosphere_states=trajectory['geosphere'],
            times=trajectory['times']
        )
        print(f"Results saved to: {save_path}")
    except Exception as e:
        print(f"Error saving results: {str(e)}")

# Mount Google Drive and save results
try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Create output directory
    output_dir = '/content/drive/MyDrive/earth_system_results'
    os.makedirs(output_dir, exist_ok=True)
    
    # Save results
    save_path = os.path.join(output_dir, 'simulation_results.npz')
    save_results(trajectory, save_path)
except Exception as e:
    print(f"Error accessing Google Drive: {str(e)}")
    print("Saving results locally instead...")
    save_results(trajectory, 'simulation_results.npz')